# [beta] NeMo Gaurdrails - Output Parser

This notebook walks through using the NeMo output parser from Nvidia, which can be used to check for hallucinations and contradictions. 

Note that this is currently a beta feature, and is subject to changes from the NeMo library.

Installation instructions and more info can be found [on the NeMo github page](https://github.com/NVIDIA/NeMo-Guardrails).

In [ ]:
! apt-get update && apt-get install -y libsndfile1 ffmpe
! pip install Cython
! pip install nemo_toolkit['all']
! pip install --upgrade langchain

In [1]:
# enable async inside jupyter
import nest_asyncio
nest_asyncio.apply()

In [2]:
from langchain.llms import OpenAI

from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.output_parsers.nemo import NeMoGaurdrailsOutputParser
from llama_index.llm_predictor import StructuredLLMPredictor
from llama_index.prompts.prompts import QuestionAnswerPrompt, RefinePrompt
from llama_index.prompts.default_prompts import (
    DEFAULT_TEXT_QA_PROMPT_TMPL,
    DEFAULT_REFINE_PROMPT_TMPL,
)

/home/loganm/miniconda3/envs/gpt_index/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

In [4]:
# setup llms
llm = OpenAI()
llm_predictor = StructuredLLMPredictor(llm=llm)

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, chunk_size_limit=512
)

The NeMo gaurdrails parser provides a few options, mainly for checking for hallucinations and if the context facts support the answer.

Note that only the `OpenAI` LLM class is supported by NeMo currently.

In [5]:
# nemo setup
nemo_parser = NeMoGaurdrailsOutputParser(
    llm=llm,
    check_hallucination=True,
    check_facts=True,
    remove_failed_facts=False
)
qa_prompt = QuestionAnswerPrompt(DEFAULT_TEXT_QA_PROMPT_TMPL, output_parser=nemo_parser)
refine_prompt = RefinePrompt(DEFAULT_REFINE_PROMPT_TMPL, output_parser=nemo_parser)

In [6]:
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

In [7]:
query_engine = index.as_query_engine(
    text_qa_template=qa_prompt, refine_template=refine_prompt
)

In [9]:
response = query_engine.query("Make up a fake story about Interleaf.")
print(response)



> Entering new LLMChain chain...

> Finished chain.

Interleaf was a small software company founded by three college friends in the late 1990s. They were ambitious and wanted to make a big impact in the software industry, so they decided to focus on creating a revolutionary new product. 

They worked hard for months, and eventually their hard work paid off: they created a powerful document editing and publishing software. It was one of the first of its kind and was quickly adopted by businesses and individuals alike. 

The company grew quickly and soon they had offices around the world. They had a team of dedicated engineers and programmers working to constantly improve their software. They also had a team of product managers, marketers, and customer service representatives to help spread the word and ensure customer satisfaction. 

Interleaf became a major player in the software industry and their software was used by millions of people around the world. They had a long list of feat